In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers
import matplotlib.pyplot as plt
# import configparser
import json
import numpy as np
import pathlib
from os.path import join as ospj
import os
from shutil import copy

Init Plugin
Init Graph Optimizer
Init Kernel


```bash
docker container rm sl-$SLNUM ; bash ./swarm-learning/bin/run-sl        \
    --name=sl-$SLNUM                         \
    --network sl-net             \
    --sl-platform=TF                   \
    --host-ip=sl-$SLNUM                \
    --sn-ip=sn-$SNNUM                   \
    --data-dir="/home/yudonghan/storage/SwarmSense/CIFAR/SplitResults-1-1-1/$SLNUM"  \
    --model-dir="/home/yudonghan/storage/SwarmSense/CIFAR/SplitResults-1-1-1/$SLNUM"  \
    --model-program=densenet.py        \
    --apls-ip apls                 \
    --gpu=$GPU                        \
    -serverAddress spire-server            \
    -genJoinToken  \
    -e SLNUM=$SLNUM  \
    -e SNNUM=$SNNUM  \
    -e WEIGHTAGE=$WEIGHTAGE \
    -e TF_FORCE_GPU_ALLOW_GROWTH=true
```

In [2]:
tmp = [('cifar10.npz', 'SplitResults-'), ('imb_cifar10.npz', 'SplitResults_LT-')]
DS_PATH, DIR_BASE_NAME = tmp[0]

In [3]:
loaded = np.load(DS_PATH)
train_images = loaded['train_images']
train_labels = loaded['train_labels']
test_images = loaded['test_images']
test_labels = loaded['test_labels']

In [4]:
train = list(zip(train_images, train_labels))
test = list(zip(test_images, test_labels))

In [5]:
np.random.seed(113)
np.random.shuffle(train)

In [6]:
def work(lst, test_images, test_labels, weightage = [1,1,1], dir_base_name = 'SplitResults-'):
    dir_name = dir_base_name + '-'.join(map(str, weightage))
    if os.path.exists(dir_name):
        print(dir_name, 'exists')
        os.system(f'rm -rf {dir_name}')
    os.mkdir(dir_name)

    weightage = np.array(weightage)
    weightage = weightage / weightage.sum()
    weightage = np.cumsum(weightage)
    weightage = len(lst) * weightage
    weightage = np.around(weightage)
    weightage = weightage.astype(np.int32)
    weightage = weightage[:-1]
    print(weightage)
    
    images, labels = zip(*lst)
    image_lst = np.split(images, weightage)
    label_lst = np.split(labels, weightage)
    assert all(len(x) == len(y) for x,y in zip(image_lst, label_lst))

    for i, (train_images, train_labels) in enumerate(zip(image_lst, label_lst)):
        path = ospj(dir_name, str(i+1))
        os.mkdir(path)
        np.savez_compressed(
            ospj(path, 'cifar10.npz'),
            train_images=train_images,
            train_labels=train_labels,
            test_images=test_images,
            test_labels=test_labels
        )
        print('saved', path, 'with', len(train_images), 'train images and', len(test_images), 'test images')
    return dir_name

In [7]:
dir_name = work(train, test_images, test_labels, [1,2,3], dir_base_name=DIR_BASE_NAME)
dir_name

[ 3405 10216]
saved SplitResults_LT-1-2-3/1 with 3405 train images and 10000 test images
saved SplitResults_LT-1-2-3/2 with 6811 train images and 10000 test images
saved SplitResults_LT-1-2-3/3 with 10215 train images and 10000 test images


'SplitResults_LT-1-2-3'

In [8]:
for x in [1,2,3]:
    # copy('main.py', 'SplitResults-1-1-1/{0}/main.py'.format(x))
    copy('densenet.py', f'{dir_name}/{x}/densenet.py')
    # json.dump(config, open('SplitResults-1-1-1/{0}/config.json'.format(x), 'w'))
os.system('rsync -avP --exclude-from ~/.rsync_exclude ./ sei:storage/SwarmSense/CIFAR/')

building file list ... 
148 files to consider
./
densenet.py
       11116 100%    9.93MB/s    0:00:00 (xfer#1, to-check=144/148)
imb_data.ipynb
       25675 100%   24.49MB/s    0:00:00 (xfer#2, to-check=142/148)
split_ds.ipynb
        6791 100%    6.48MB/s    0:00:00 (xfer#3, to-check=138/148)
SplitResults_LT-1-1-1/1/
SplitResults_LT-1-1-1/2/
SplitResults_LT-1-1-1/3/
SplitResults_LT-1-2-3/
SplitResults_LT-1-2-3/1/
SplitResults_LT-1-2-3/1/cifar10.npz
    37933268 100%   94.95MB/s    0:00:00 (xfer#4, to-check=93/148)
SplitResults_LT-1-2-3/1/densenet.py
       11116 100%   28.42kB/s    0:00:00 (xfer#5, to-check=92/148)
SplitResults_LT-1-2-3/2/
SplitResults_LT-1-2-3/2/cifar10.npz
    47626412 100%   53.12MB/s    0:00:00 (xfer#6, to-check=90/148)
SplitResults_LT-1-2-3/2/densenet.py
       11116 100%   12.70kB/s    0:00:00 (xfer#7, to-check=89/148)
SplitResults_LT-1-2-3/3/
SplitResults_LT-1-2-3/3/cifar10.npz
    57330643 100%   40.11MB/s    0:00:01 (xfer#8, to-check=87/148)
SplitResults_LT-1

0

In [ ]:
config = json.load(open('config.json'))
config['base_name'] = config['base_name'].replace('CL', 'SL')
config['sl'] = True
print(config)

In [ ]:
os.system('jupyter nbconvert --to python main.ipynb')

In [ ]:
dir_name = work(train, test_images, test_labels, [1,3,9])


In [ ]:
config = json.load(open('config.json'))
config['base_name'].replace('CL', 'LL')
print(config)

In [ ]:
for x in [1,2,3]:
    copy('main.py', 'SplitResults-1-3-9/{0}/main.py'.format(x))
    copy('densenet.py', 'SplitResults-1-3-9/{0}/densenet.py'.format(x))
    json.dump(config, open('SplitResults-1-3-9/{0}/config.json'.format(x), 'w'))